# VacationPy

In [65]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
# Import API key
from api_keys import g_key

In [66]:
#Load the csv exported in WeatherPy to a DataFrame
csv_file = os.path.join("city_data.csv")
city_df = pd.read_csv(csv_file)
city_df.head()

,City,lat,lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Albany,42.60,-73.97,73.00,45,16,7.47,US,1600898722
1,Rikitea,-23.12,-134.97,72.79,73,1,11.99,PF,1600898722
2,Malbork,54.04,19.03,64.40,63,50,7.63,PL,1600898723
3,Sivaki,52.63,126.75,49.78,96,100,9.80,RU,1600898723
4,Bousso,10.48,16.71,73.76,89,100,4.97,TD,1600898723


### Humidity Heatmap

In [67]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [68]:
#Use the Lat and Lng as locations and Humidity as the weight
locations = city_df[["lat", "lng"]].astype(float)

humidity = city_df["Humidity"].astype(float)

In [69]:
#Add Heatmap layer to map
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
#Narrow down the cities to fit weather conditions
hotel_df = city_df.loc[(city_df["Max Temp"] < 80)| 
                       (city_df["Max Temp"] > 70) |
                       (city_df["Wind Speed"]< 10) | 
                       (city_df["Clouds"] == 0),:]

#Drop any rows will null values
hotel_df.dropna()

,City,lat,lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date
0,Albany,42.60,-73.97,73.00,45,16,7.47,US,1600898722
1,Rikitea,-23.12,-134.97,72.79,73,1,11.99,PF,1600898722
2,Malbork,54.04,19.03,64.40,63,50,7.63,PL,1600898723
3,Sivaki,52.63,126.75,49.78,96,100,9.80,RU,1600898723
4,Bousso,10.48,16.71,73.76,89,100,4.97,TD,1600898723
...,...,...,...,...,...,...,...,...,...
523,Ambon City,-3.70,128.18,75.20,94,40,3.36,ID,1600898819
524,Iqaluit,63.75,-68.51,35.01,83,90,21.92,CA,1600898734
525,San Jose,37.34,-121.89,84.00,39,20,11.41,US,1600898705
526,Acapulco de Juárez,16.86,-99.89,87.80,74,75,8.05,MX,1600898819


### Hotel Map

In [48]:
#Create an empty list to store the names of the hotel
hotel_name= []

#iterate through latitude and longitude to get location
for ind, row in hotel_df.iterrows():
    lat = row["lat"]
    lng = row["lng"]
    target_coordinates = f"{lat}, {lng}"

    #set the parameters of query
    params = {
        "location": target_coordinates,
        "keyword" :"hotel",
        "radius": 5000,
        "type": "hotel",
        "key": g_key
        }

    #from the base url run a request using the parameters
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    
    #convert response to json and pretty print to check the data
    hotel_data = response.json()
    #print(json.dumps(hotel_data, indent=4, sort_keys=True)) 
    
    try:
        hotel_name.append(hotel_data['results'][0]["name"])
    except IndexError:
        hotel_name.append(np.nan)
    
hotel_name

[nan,
 'Pension Bianca & Benoit',
 'Hotel Grot',
 'Sivaki',
 nan,
 'Sberbank, Bankomat',
 'Hotel Diponegoro',
 'Hotel Qaqortoq',
 'Hilton Garden Inn Kauai Wailua Bay',
 nan,
 'Hôtel CTK',
 'Las Hayas Ushuaia Resort',
 'Top of the World Hotel',
 "Ellie's Villa",
 'Graciosa Resort - Biosphere Island Hotel',
 'Birkenhead House',
 'Palace of Zinos',
 'Shangri-La Hotel',
 'Galapagos Morning Glory',
 nan,
 'La Cueva del Pirata',
 'The Explorer Hotel',
 'Paradise First Hotel',
 'Les Embruns Du Baril',
 'Comfort Inn & Suites',
 'Carnarvon Motel',
 'Promenade Hotel',
 'Best Western Plus Quarterdecks Retreat',
 'Grand Naniloa Hotel Hilo - a DoubleTree by Hilton',
 'Presidential Inn & Suites',
 'Splash Guest House',
 'Atlanticview Cape Town Boutique Hotel',
 'Hotel Boutique Restaurant Kran Kreen',
 'Central Palace Hotel - Castro (PR)',
 nan,
 'Nasta Apartment',
 'Hotel National',
 'Hotel Los Caracoles',
 'Welk Resorts Cabo San Lucas',
 'Pacific Resort Rarotonga',
 'Steppe hotel',
 "Gostinitsa ''y

In [62]:
#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = hotel_name

#Drop any rows will null values
hotel_df =hotel_df.dropna()
hotel_df

,City,lat,lng,Max Temp,Humidity,Clouds,Wind Speed,Country,Date,Hotel Name
1,Rikitea,-23.12,-134.97,72.79,73,1,11.99,PF,1600898722,Pension Bianca & Benoit
2,Malbork,54.04,19.03,64.40,63,50,7.63,PL,1600898723,Hotel Grot
3,Sivaki,52.63,126.75,49.78,96,100,9.80,RU,1600898723,Sivaki
5,Peleduy,59.63,112.74,42.26,87,0,2.86,RU,1600898723,"Sberbank, Bankomat"
6,Gorontalo,0.54,123.06,74.55,87,93,2.55,ID,1600898723,Hotel Diponegoro
...,...,...,...,...,...,...,...,...,...,...
523,Ambon City,-3.70,128.18,75.20,94,40,3.36,ID,1600898819,Orchid Hotel
524,Iqaluit,63.75,-68.51,35.01,83,90,21.92,CA,1600898734,Frobisher Inn
525,San Jose,37.34,-121.89,84.00,39,20,11.41,US,1600898705,Four Points by Sheraton San Jose Downtown
526,Acapulco de Juárez,16.86,-99.89,87.80,74,75,8.05,MX,1600898819,Encanto Acapulco


In [63]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [64]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content= hotel_info)

# Display figure                              
fig.add_layer(hotel_layer)
  
fig                                 

Figure(layout=FigureLayout(height='420px'))